# Обучение модели 

In [2]:
import tensorflow as tf

import numpy as np
import os
# from utils import read_wav, extract_feats, read_dataset, batch, decode
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from IPython.core.display import HTML
from scipy.signal import spectrogram



import scipy.io.wavfile as wav
from python_speech_features import fbank, mfcc
import time

from itertools import izip_longest as zip_longest

from keras.layers import LSTM, Dense, Convolution1D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed, Bidirectional

%matplotlib inline

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
def decode(d, mapping):
    """Decode."""
    shape = d.dense_shape
    batch_size = shape[0]
    ans = np.zeros(shape=shape, dtype=int)
    seq_lengths = np.zeros(shape=(batch_size, ), dtype=np.int)
    for ind, val in zip(d.indices, d.values):
        ans[ind[0], ind[1]] = val
        seq_lengths[ind[0]] = max(seq_lengths[ind[0]], ind[1] + 1)
    ret = []
    for i in range(batch_size):
        ret.append("".join(map(lambda s: mapping[s], ans[i, :seq_lengths[i]])))
    return ret


In [4]:
def list_2d_to_sparse(list_of_lists):
    """Convert python list of lists to a [tf.SparseTensorValue](https://www.tensorflow.org/api_docs/python/tf/SparseTensorValue).

    Args:
        list_of_lists: list of lists to convert.

    Returns:
        tf.SparseTensorValue which is a namedtuple (indices, values, shape) where:

            * indices is a 2-d numpy array with shape (sum_all, 2) where sum_all is a
            sum over i of len(l[i])

            * values is a 1-d numpy array with shape (sum_all, )

            * shape = np.array([len(l), max_all]) where max_all is a max over i of
            len(l[i])

        Also, the following is true: for all i values[i] ==
        list_of_lists[indices[i][0]][indices[i][1]]

    """
    indices, values = [], []
    for i, sublist in enumerate(list_of_lists):
        for j, value in enumerate(sublist):
            indices.append([i, j])
            values.append(value)
    dense_shape = [len(list_of_lists), max(map(len, list_of_lists))]
    return tf.SparseTensorValue(indices=np.array(indices),
                                values=np.array(values),
                                dense_shape=np.array(dense_shape))


In [5]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33}

inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))
inv_mapping[34]='<пробел>'

In [6]:
# Каталог с базой аудио
voxforge_dir = './Voxforge'

In [5]:
speaker_folders = os.listdir(voxforge_dir)

In [6]:
speaker_folders

['anonymous-20110505-ane',
 'uvgeek-20150608-etf',
 '1981-20120705-rjp',
 'vad-20160619-ykt',
 'anonymous-20150112-pcj',
 'anonymous-20120820-vyn',
 'h12-20110505-tyk',
 'anonymous-20110508-hhh',
 'anonymous-20110504-ewo',
 'vbuoc-20091116-bkd',
 'alexmo-20111111-ydn',
 'DeMusi-20150707-rid',
 'anonymous-20100410-kwc',
 'anonymous-20110507-lmh',
 'dims-20100915-lbd',
 'snoop-20110506-lzu',
 'anonymous-20110519-nyf',
 'evgeny_ta-20090910-nvp',
 'dmitriytim-20160401-fnw',
 'anonymous-20141010-jln',
 'Sergey-20080502-ddg',
 'yalexand-20110511-rkv',
 'anonymous-20120718-smp',
 'anonymous-20110504-lmr',
 'Nick-20110505-erb',
 'evgeny_ta-20090922-wma',
 'ppc-20130614-xbd',
 'snoop-20110506-gtp',
 'anonymous-20110504-bae',
 'Mikola-20111113-yyf',
 'anonymous-20110502-wqi',
 'anonymous-20150721-zdo',
 'anonymous-20130106-gyj',
 'anonymous-20130106-tau',
 'anonymous-20110504-btq',
 'anonymous-20121028-ifr',
 'anonymous-20150919-edt',
 'anonymous-20091118-psk',
 'anonymous-20110508-qqw',
 'snoop

In [7]:
def list_files_for_speaker(speaker, folder):
    """
    Generates a list of wav files for a given speaker from the voxforge dataset.
    Args:
        speaker: substring contained in the speaker's folder name, e.g. 'Aaron'
        folder: base folder containing the downloaded voxforge data

    Returns: list of paths to the wavfiles
    """

    speaker_folders = [d for d in os.listdir(folder) if speaker in d]
    wav_files = []

    for d in speaker_folders:
        for f in os.listdir(os.path.join(folder, d, 'wav')):
            wav_files.append(os.path.abspath(os.path.join(folder, d, 'wav', f)))

    return wav_files

In [71]:
def extract_features_and_targets(wav_file):
    
#     обработка текст
    txt_file = wav_file.replace('/wav/', '/txt/').replace('.wav', '.txt')
    try:
        f = open(txt_file, 'rb')
    except IOError as e:
#         print(u'не удалось открыть файл')
        return 0,0,0,0
    else:
        with f:
#     with open(txt_file, 'rb') as f:
            for line in f.readlines():
                if line[0] == ';':
                    continue

            # Get only the words between [a-z] and replace period for none
            
            original = ' '.join(unicode(line,'utf-8').strip().lower().split(' ')).replace('.', '').replace("'", '').replace('-', '').replace(',','')
            targets = original.replace(' ', '  ')
            targets = targets.split(' ')
            stroka=[34]

            for i in targets:
                i1=i.encode("UTF-8")
                for j in range(0,len(i1),2):
                    stroka.append(vocabulary.get(i1[j:j+2],34))
                if stroka[-1] != 34:
                    stroka.append(34)
#                     обработка звука
            fs, audio = wav.read(wav_file)

            features = mfcc(audio, samplerate=fs, lowfreq=50)

            mean_scale = np.mean(features, axis=0)
            std_scale = np.std(features, axis=0)

            features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]

            seq_len = features.shape[0]
            return features, stroka, seq_len, original
   
            


In [9]:
# Подготавливаем данные для обучения - примерно 3Гб
X = []
y = []
for j in speaker_folders:
    wav_files = list_files_for_speaker(j, voxforge_dir)
    for i in wav_files:
        features, stroka, seq_len, original=extract_features_and_targets(i)
        if seq_len !=0 :
            X.append(features)
            y.append(stroka)

In [10]:
len(y)

8952

In [68]:
def decode1(stroka,inv_mapping):
    ret = []
    for i in range(len(stroka)):
         ret.append("".join( inv_mapping.get(int(stroka[i]),34)))
    for i in range(len(ret)):
        print(str(ret[i])),
    print('')

In [69]:
decode1(stroka,inv_mapping)

<пробел> о д н и <пробел> и <пробел> т е <пробел> ж е <пробел> с е м е н а <пробел> о с е д а ю т <пробел> н а <пробел> м а р с <пробел> и <пробел> н а <пробел> з е м л ю <пробел> н а <пробел> в с е <пробел> м и р и а д ы <пробел> о с т ы в а ю щ и х <пробел> з в ё з д <пробел> 


In [32]:
def batch(X_train, y_train, batch_size):
    num_features = X_train[0].shape[1]
    n = len(X_train)
    perm = np.random.permutation(n)
    for batch_ind in np.resize(perm, (n // batch_size, batch_size)):
        X_batch, y_batch = [X_train[i] for i in batch_ind], [y_train[i] for i in batch_ind]
        sequence_lengths = list(map(len, X_batch))
        X_batch_padded = np.array(list(zip_longest(*X_batch, fillvalue=np.zeros(num_features)))).transpose([1, 0, 2])
        yield X_batch_padded, sequence_lengths, list_2d_to_sparse(y_batch), y_batch

# Модель нейронной сети

In [35]:
graph = tf.Graph()
with graph.as_default():
    input_X = tf.placeholder(tf.float32, shape=[None, None, 13],name="input_X")
    labels = tf.sparse_placeholder(tf.int32)
    seq_lens = tf.placeholder(tf.int32, shape=[None],name="seq_lens")


    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2), input_shape=(None, 13)))
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2)))
    model.add(TimeDistributed(Dense(len(inv_mapping) + 2)))
    
    final_seq_lens = seq_lens

    logits = model(input_X)
    logits = tf.transpose(logits, [1, 0, 2])

    ctc_loss = tf.reduce_mean(tf.nn.ctc_loss(labels, logits, final_seq_lens))
    # ctc_greedy_decoder? merge_repeated=True
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, final_seq_lens)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), labels))

    train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(ctc_loss)

In [ ]:
num_epochs=181
epoch_save_step=10

In [ ]:
# Обучаемся в начале на коротких данных 
y_train1=[]
X_train1=[]
for i in range(len(y)):
    if len(y[i]) <75:
        y_train1.append(y[i])
        X_train1.append(X[i])

In [73]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 100):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,

                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

[i] LOADING checkpoint checkpoint1/ctc.ckpt-80
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-80
[i] start from epoch 81
(81, 8.6621685, 0.042762309)
<пробел>желание<пробел>у<пробел>тебя<пробел>именно<пробел>такое<пробел> <пробел>раздался<пробел>железный<пробел>голожу<пробел>ни<пробел>ка<пробел>спиной<пробел> <пробел>без<пробел>окна<пробел>и<пробел>безчасов<пробел>о<пробел>времене<пробел>трудно<пробел>судить<пробел> <пробел>в<пробел>один<пробел>из<пробел>глазков<пробел>входил<пробел>прямой<пробел>ослепительный<пробел>луч<пробел>солнца<пробел> <пробел>так<пробел>же<пробел>как<пробел><пробел>гооворить<пробел>с<пробел>ней<пробел>об<пробел>это<пробел> <пробел>когда<пробел>есть<пробел>машина<пробел>жить<пробел>в<пробел>правдо<пробел>левче<пробел> <пробел>ах<пробел>боже<пробел>мой<пробел>какие<пробел>могли<пробел>быль<пробел>дела<пробел>важнее<пробел>катюшиной<пробел>любви<пробел> <пробел>говорила<пробел>она<пробел>без<пробел>всякого<пробел>выраженея<пробел> <пробел>через<про

(87, 6.0689678, 0.025693689)
<пробел>аванс<пробел>шеть<пробел>фельетонов<пробел>по<пробел>двести<пробел>строк<пробел>чек<пробел>можете<пробел>учесть<пробел>в<пробел>стокгольме<пробел> <пробел>так<пробел>быстро<пробел>и<пробел>на<пробел>моих<пробел>глазах<пробел> <пробел>затем<пробел>уходит<пробел>вслед<пробел>за<пробел>ней<пробел> <пробел>вва<пробел>три<пробел>он<пробел>не<пробел>помнил<пробел>сколько<пробел> <пробел>аванс<пробел>шесть<пробел>фельлетонов<пробел>по<пробел>двести<пробел>строк<пробел>чек<пробел>можете<пробел>учесть<пробел>в<пробел>стокгольме<пробел> <пробел>мне<пробел>в<пробел>его<пробел>обществ<пробел>просто<пробел>скужно<пробел> <пробел>реди<пробел>прочих<пробел>рядон<пробел>с<пробел>усачом<пробел>стоит<пробел>в<пробел>профиль<пробел>наш<пробел>зубр<пробел> <пробел>что<пробел>у<пробел>этой<пробел>барышни<пробел>было<пробел>в<пробел>голове<пробел>не<пробел>пойму<пробел>до<пробел>сих<пробел>пор<пробел> <пробел>он<пробел>будет<пробел>жить<пробел>у<пробел>нас<пробел> <пробе

(94, 6.7858377, 0.03656017)
<пробел>свет<пробел>проникал<пробел>в<пробел>неё<пробел>сквозь<пробел>забранные<пробел>стеклом<пробел>отверстия<пробел>сводчатого<пробел>купала<пробел> <пробел>просто<пробел>ре<пробел>шил<пробел>спроснсь<пробел>на<пробел>всякий<пробел>случа<пробел> <пробел>что<пробел>ты<пробел>думаешь<пробел>доедет<пробел>токолесо<пробел>еслиб<пробел>случи<пробел>лось<пробел>в<пробел>москву<пробел>или<пробел>не<пробел>доедеть<пробел> <пробел>сначала<пробел>думали<пробел>чтоэто<пробел>следы<пробел>бурь<пробел>в<пробел>магнитных<пробел>полях<пробел>земли<пробел> <пробел>аванс<пробел>шесть<пробел>фельетонов<пробел>по<пробел>двести<пробел>строк<пробел>чек<пробел>можете<пробел>учесть<пробел>в<пробел>стокгольме<пробел> <пробел>он<пробел>ее<пробел>забыл<пробел>он<пробел>любит<пробел>другую<пробел> <пробел>ведь<пробел>ад<пробел>и<пробел>есть<пробел>моё<пробел>безнадёжное<пробел>одиночество<пробел>распростёртое<пробел>в<пробел>вечной<пробел>тьме<пробел> <пробел>ведь<пробел>ад<пробел>

[i] SAVING snapshot ctc
(100, 4.2735562, 0.018387472)
<пробел>осенний<пробел>ветер<пробел>над<пробел>катей<пробел>лежащей<пробел>в<пробел>земле<пробел>под<пробел>крестом<пробел> <пробел>так<пробел>я<пробел>этого<пробел>никогда<пробел><пробел>не<пробел>узнал<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>труп<пробел>влетит<пробел>в<пробел>её<пробел>огненные<пробел>океаны<пробел> <пробел>они<пробел>минули<пробел>тёмный<пробел>вестибюль<пробел>и<пробел>вошли<пробел>в<пробел>многоугольную<пробел>высокую<пробел>залу<пробел> <пробел>завтра<пробел>они<пробел>у<пробел>вас<пробел>будут<пробел> <пробел>за<пробел>что<пробел>и<пробел>почему<пробел>шли<пробел>за<пробел>него<пробел>такие<пробел> <пробел>лицо<пробел>его<пробел>в<пробел>эту<пробел>минуту<пробел>казалось<пробел>утомлёным<пробел>лоб<пробел>сморщился<пробел> <пробел>зеленый<пробел>цвет<пробел>еи<пробел>собено<пробел>к<пробел>лицу<пробел> <пробел>он<пробел>поцеловал<пробел>нее<пробел>не<пробел>говоря<пр

(106, 3.5940754, 0.017911507)
<пробел>если<пробел>это<пробел>можно<пробел>азвать<пробел>работой<пробел> <пробел>в<пробел>один<пробел>из<пробел>глазков<пробел>ввходил<пробел>прямой<пробел>ослепительный<пробел>луч<пробел>солнца<пробел> <пробел>лицо<пробел>его<пробел>в<пробел>эту<пробел>минуту<пробел>казалось<пробел>утомлёным<пробел>лоб<пробел>сморщился<пробел> <пробел>что<пробел>же<пробел>может<пробел>быть<пробел>арля<пробел>и<пробел>вспрямь<пробел>ждёт<пробел>удача<пробел> <пробел>дескать<пробел>господин<пробел>лейтенант<пробел>хочете<пробел>пить<пробел> <пробел>мне<пробел>в<пробел>его<пробел>обществе<пробел>просто<пробел>скужно<пробел> <пробел>то<пробел>что<пробел>призошло<пробел>в<пробел>действительнсти<пробел> <пробел>и<пробел>доктор<пробел>взглянул<пробел>на<пробел>часы<пробел> <пробел>он<пробел>просто<пробел>не<пробел>мог<пробел>бы<пробел>соверщить<пробел>такой<пробел>поступок<пробел> <пробел>лось<пробел>с<пробел>трудом<пробел>оторвался<пробел>от<пробел>этого<пробел>зрелища<пробел>

...

[i] SAVING snapshot ctc
(160, 1.1014397, 0.0031062551)
<пробел>люблю<пробел>одного<пробел>мужчину<пробел>вместо<пробел>того<пробел>чтобы<пробел>сказать<пробел> <пробел>лось<пробел>с<пробел>трудом<пробел>оторвался<пробел>от<пробел>этого<пробел>зрелища<пробел>живоносного<пробел>огня<пробел>вселенной<пробел> <пробел>ведь<пробел>ад<пробел>и<пробел>есть<пробел>моё<пробел>безнадёжное<пробел>одиночество<пробел>распростёртоев<пробел>вечной<пробел>тьме<пробел> <пробел>обоз<пробел>отобьём<пробел>и<пробел>к<пробел>вечеру<пробел>мы<пробел>вёрст<пробел>уж<пробел>за<пробел>восемьдесят<пробел> <пробел>со<пробел>спокойным<пробел>мужеством<пробел>скайлс<пробел>ожидал<пробел>всего<пробел>в<пробел>этом<пробел>безумном<пробел>городе<пробел> <пробел>вы<пробел>верно<пробел>знаете<пробел>что<пробел>это<пробел>стоит<пробел> <пробел>в<пробел>один<пробел>из<пробел>глазков<пробел>входил<пробел>прямой<пробел>ослепительный<пробел>луч<пробел>солнца<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>ок

(166, 0.49753883, 0.00053513516)
<пробел>хочет<пробел>он<пробел>того<пробел>или<пробел>нет<пробел> <пробел>что<пробел>он<пробел>желал<пробел>было<пробел>мучительно<пробел>для<пробел>него<пробел> <пробел>так<пробел>с<пробел>бубен<пробел> <пробел>учился<пробел>на<пробел>медные<пробел>деньги<пробел>с<пробел>двенадцати<пробел>лет<пробел>сам<пробел>их<пробел>зарабатываю<пробел> <пробел>ты<пробел>же<пробел>все<пробел>прекрасно<пробел>слышал<пробел> <пробел>есть<пробел>океан<пробел>он<пробел>здесь<пробел>рядом<пробел> <пробел>пичура<пробел>и<пробел>даже<пробел>просто<пробел>пичук<пробел> <пробел>ах<пробел>боже<пробел>мой<пробел>какие<пробел>могли<пробел>быть<пробел>дела<пробел>важнее<пробел>катюшиной<пробел>любви<пробел> <пробел>над<пробел>этой<пробел>машиной<пробел>он<пробел>ткнул<пробел>трубкой<пробел>в<пробел>сторону<пробел>лесов<пробел>работаю<пробел>давно<пробел> <пробел>он<пробел>целует<пробел>её<пробел>в<пробел>щёки<пробел>в<пробел>лоб<пробел>в<пробел>закрытые<пробел>веки<пробел> Fri A

(173, 0.35636896, 0.00015873017)
<пробел>в<пробел>угольной<пробел>из<пробел>этих<пробел>лавочек<пробел>или<пробел>лучше<пробел>в<пробел>окне<пробел>помещался<пробел>сбитенщик<пробел> <пробел>но<пробел>сегодня<пробел>все<пробел>казалось<пробел>было<пробел>против<пробел>меня<пробел> <пробел>в<пробел>обычно<пробел>мутных<пробел>зрачках<пробел>кондитера<пробел>блестели<пробел>теперь<пробел>зелёные<пробел>огоньки<пробел> <пробел>сначала<пробел>думали<пробел>что<пробел>это<пробел>следы<пробел>бурь<пробел>в<пробел>магнитных<пробел>полях<пробел>земли<пробел> <пробел>надоело<пробел>мало<пробел>толку<пробел>да<пробел>уж<пробел>и<пробел>мужикам<пробел>махновщина<пробел>эта<пробел>стала<пробел>надоедать<пробел> <пробел>мы<пробел>втянули<пробел>его<пробел>и<пробел>он<пробел>упал<пробел>разбил<пробел>земную<пробел>кору<пробел>отклонил<пробел>полюсы<пробел> <пробел>надоело<пробел>мало<пробел>толку<пробел>да<пробел>уж<пробел>и<пробел>мужикам<пробел>махновщина<пробел>эта<пробел>стала<пробел>надоедать<п

(179, 0.2845543, 0.00014925373)
<пробел>такие<пробел>что<пробел>мы<пробел>и<пробел>вообразить<пробел>не<пробел>можем<пробел> <пробел>надоело<пробел>мало<пробел>толку<пробел>да<пробел>уж<пробел>и<пробел>мужикам<пробел>махновщина<пробел>эта<пробел>стала<пробел>надоедать<пробел> <пробел>в<пробел>один<пробел>из<пробел>глазков<пробел>входил<пробел>прямой<пробел>ослепительный<пробел>луч<пробел>солнца<пробел> <пробел>свет<пробел>проникал<пробел>в<пробел>неё<пробел>сквозь<пробел>забранные<пробел>стеклом<пробел>отверстия<пробел>сводчатого<пробел>купола<пробел> <пробел>лось<пробел>кинулся<пробел>к<пробел>реостатам<пробел>попали<пробел>алексей<пробел>иванович<пробел>притягиваемся<пробел>падаем<пробел> <пробел>но<пробел>погода<пробел>была<пробел>такая<пробел>что<пробел>он<пробел>мог<пробел>вотвот<пробел>пойти<пробел> <пробел>в<пробел>деревню<пробел>ехать<пробел>отец<пробел>с<пробел>матерью<пробел>померли<пробел>братья<пробел>убиты<пробел>земля<пробел>заброшена<пробел> <пробел>они<пробел>минули<про

KeyboardInterrupt: 

Обучаемся на средних данных (длина предложения меньше 181 символа)

In [74]:

y_train1=[]
X_train1=[]
for i in range(len(y)):
    if len(y[i]) <181:
        y_train1.append(y[i])
        X_train1.append(X[i])


0

In [75]:
num_epochs=231
epoch_save_step=10

In [ ]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 50):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,

                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

[i] LOADING checkpoint checkpoint1/ctc.ckpt-180
INFO:tensorflow:Restoring parameters from checkpoint1/ctc.ckpt-180
[i] start from epoch 181
(181, 75.390259, 0.25192869)
<пробел>а<пробел>свявын<пробел>поосаи<пробел>з<пробел>омпа<пробел>дестяотишесам<пробел> <пробел>но<пробел>ме<пробел>так<пробел>ок<пробел>теля<пробел> <пробел>я<пробел>бысделасьбода<пробел> <пробел>семьлет<пробел>ме<пробел>ни<пробел>пуля<пробел>ни<пробел>штых<пробел>не<пробел>моглиссебить<пробел> <пробел>и<пробел>не<пробел>номы<пробел>что<пробел><пробел>ну<пробел>мелн<пробел>это<пробел>праго<пробел> <пробел>тогдалось<пробел>вышал<пробел>надвор<пробел>первы<пробел>разаети<пробел>семнее<пробел> <пробел>унцелует<пробел>ё<пробел>в<пробел>щёкиглокзакротыевеке<пробел> <пробел>сквозь<пробел>раскорытый<пробел>пален<пробел>ке<пробел>ворот<пробел>были<пробел>ведною<пробел>багролы<пробел>полосым<пробел>заката<пробел>итклубы<пробел>туч<пробел>поднявшихся<пробел>с<пробел>моря<пробел> <пробел>аланц<пробел>шесть<пробел>пелелдунопотвест

(186, 30.004221, 0.12891933)
<пробел>несколько<пробел>лизний<пробел>точек<пробел>и<пробел>полукружий<пробел>разбросано<пробел>к<пробел>западу<пробел>и<пробел>востоку<пробел>о<пробел>еэтой<пробел>ээкваторильной<пробел>группы<пробел> <пробел>вмолодости<пробел>я<пробел>курил<пробел>десятипфениговое<пробел>сегары<пробел>потом<пробел>когдо<пробел>стал<пробел>з<пробел>рабатыват<пробел>не<пробел>много<пробел>больши<пробел>доводцатип<пробел>фениговое<пробел>оза<пробел>тем<пробел>несколько<пробел>десятко<пробел>в<пробел>флет<пробел>шестидеситипфениговые<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>труп<пробел>влетет<пробел>в<пробел>еёогненные<пробел>океаны<пробел> <пробел>они<пробел>минули<пробел>тёмный<пробел>встибюль<пробел>пошлив<пробел>могогольну<пробел>высоккую<пробел>залу<пробел> <пробел>апванс<пробел>шесть<пробел>фельитонов<пробел>по<пробел>двести<пробел>строк<пробел>чек<пробел>можете<пробел>учесть<пробел>стоккольме<пробел> <пробел>скайльс<пробел>тож

(191, 29.011711, 0.10632061)
<пробел>этоне<пробел>входит<пробел>в<пробел>мони<пробел>планы<пробел> <пробел>сначала<пробел>думали<пробел>что<пробел>это<пробел>следы<пробел>бурь<пробел>в<пробел>магнитных<пробел>полях<пробел>землин<пробел> <пробел>учился<пробел>на<пробел>медные<пробел>деньги<пробел><пробел>двенадцати<пробел>летьсам<пробел>ихе<пробел>зарабатываю<пробел> <пробел>что<пробел>у<пробел>этой<пробел>барышьни<пробел>было<пробел>в<пробел>голве<пробел>не<пробел>пойму<пробел>до<пробел>сих<пробел>пор<пробел> <пробел>шли<пробел>с<пробел>набеженой<пробел>бежали<пробел>из<пробел>перееулков<пробел>бубнили<пробел>сбивались<пробел>в<пробел>кучки<пробел>лежали<пробел>на<пробел>чахлой<пробел>праве<пробел>поглядывали<пробел>на<пробел>низкое<пробел>солнце<пробел>пустившее<пробел>сквозь<пробел>облака<пробел>широкие<пробел>лучи<пробел> <пробел>мышь<пробел>понемногу<пробел>ожила<пробел>подняла<пробел>нос<пробел>стала<пробел>шевлить<пробел>усами<пробел>умлась<пробел> <пробел>спраси<пробел>он<пробел

(196, 25.654482, 0.1161145)
<пробел>глаза<пробел>ленивае<пробел>серокари<пробел><пробел>так<пробел>же<пробел>как<пробел>у<пробел>то<пробел>жинщины<пробел>с<пробел>искоркой<пробел> <пробел>аванс<пробел>шесть<пробел>фельетонов<пробел>по<пробел>двести<пробел>строк<пробел>чек<пробел>можете<пробел>учесть<пробел>в<пробел>стокгольме<пробел> <пробел>н<пробел>тривессваи<пробел>его<пробел>как<пробел>старено<пробел>ознокуу<пробел>на<пробел>что<пробел>чичиков<пробел>раскланился<пробел>несколькой<пробел>набук<пробел>прочен<пробел>не<пробел>без<пробел>триядности<пробел> <пробел>чего<пробел>жто<пробел>раньщен<пробел>не<пробел>шла<пробел> <пробел>весе<пробел>для<пробел>меня<пробел>ни<пробел>что<пробел>родилас<пробел> <пробел>надоело<пробел>мало<пробел>току<пробел>да<пробел>уш<пробел>и<пробел>мужикам<пробел>махновщина<пробел>это<пробел>стала<пробел>надоедать<пробел> <пробел>недолго<пробел>ждать<пробел>ода<пробел>пошатнёлтся<пробел>даже<пробел>разум<пробел>единственые<пробел>отепьна<пробел>этом<пробел>ч

[i] SAVING snapshot ctc
(200, 21.861736, 0.09421172)
<пробел>от<пробел>берега<пробел>до<пробел>блежайшей<пробел>кущи<пробел>деревьев<пробел>лось<пробел>и<пробел>гусев<пробел>шли<пробел>погорелому<пробел>борому<пробел>праху<пробел>перепрыгивали<пробел>через<пробел>обсыпавшие<пробел>ся<пробел>неширокие<пробел>каналы<пробел>огибали<пробел>высохшие<пробел>прудики<пробел> <пробел>наглас<пробел>скалйльс<пробел>определил<пробел>что<пробел>яйцеобразный<пробел>аппарат<пробел>бы<пробел>не<пробел>енее<пробел>воасьми<пробел>с<пробел>половино<пробел>бедров<пробел>высоты<пробел>и<пробел>шестимывно<пробел>поперечнике<пробел> <пробел>в<пробел>один<пробел>из<пробел>глазков<пробел>входил<пробел>прямой<пробел>рослепительный<пробел>луча<пробел>солнца<пробел> <пробел>вг<пробел>лаптихстеах<пробел>узкиека<пробел>к<пробел>щели<пробел>глубакиехтверстия<пробел>окон<пробел> <пробел>через<пробел>несколько<пробел>лет<пробел>путешествия<пробел>на<пробел>марс<пробел>будет<пробел>не<пробел>более<пробел>сложно<пробел>

...

(213, 15.411602, 0.072593629)
<пробел>ведь<пробел>ад<пробел>и<пробел>есть<пробел>моё<пробел>безнадёжноединочество<пробел>распростёртое<пробел>в<пробел>вечной<пробел>тьме<пробел> <пробел>это<пробел>движения<пробел>на<пробел>минуту<пробел>пряоткрывает<пробел>её<пробел>н<пробел><пробел><пробел>акруглую<пробел>как<пробел>прекрасна<пробел>амфра<пробел>и<пробел>сквозь<пробел>чёрный<пробел>прозрачный<пробел>чувок<пробел>её<пробел>коже<пробел>просвечивые<пробел>золотистом<пробел>муаром<пробел> <пробел>я<пробел>вам<пробел>на<пробел>марсе<пробел>пригожусь<пробел>ну<пробел>очен<пробел>рад<пробел>сказал<пробел>лось<пробел>кодава<пробел>яемуруку<пробел>до<пробел>завтра<пробел> <пробел>она<пробел>завела<пробел>прядь<пробел>волнитых<пробел>волос<пробел>за<пробел>ухо<пробел>подняла<пробел>с<пробел>тротуара<пробел>корзенкуй<пробел>св<пробел>зеленаю<пробел>и<пробел>пошла<пробел>через<пробел>улицу<пробел> <пробел>о<пробел>уже<пробел>не<пробел>раз<пробел>говори<пробел>е<пробел> <пробел>сейча<пробел>же<про

(218, 13.560404, 0.059728794)
<пробел>учился<пробел>на<пробел>медные<пробел>деньги<пробел>с<пробел>двенадцати<пробел>лет<пробел>сам<пробел>их<пробел>зарабатываю<пробел> <пробел>она<пробел>завела<пробел>прядь<пробел>олнистых<пробел>воло<пробел>за<пробел>уо<пробел>подняла<пробел>с<пробел>тротуара<пробел>корзинку<пробел>с<пробел>зелеью<пробел>и<пробел>пошла<пробел>че<пробел>рез<пробел>улицу<пробел> <пробел>ва<пробел>наш<пробел>век<пробел>таких<пробел>не<пробел>мало<пробел> <пробел>сквозь<пробел>балки<пробел>лесов<пробел>поблескивала<пробел>металическая<пробел>с<пробел>частой<пробел>клёпкой<пробел>повехность<пробел>сферического<пробел>тела<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>труп<пробел><пробел>влетит<пробел>в<пробел>её<пробел>огненные<пробел>океаны<пробел> <пробел>та<пробел>к<пробел>же<пробел>как<пробел>и<пробел>говорить<пробел>сне<пробел>обаэтом<пробел> <пробел>небольшая<пробел>тверца<пробел>в<пробел>нём<пробел>была<пробел>при<пробел>отворен

(223, 18.366533, 0.072855324)
<пробел>утаки<пробел><пробел>женщин<пробел>глаза<пробел>говрят<пробел>больше<пробел>чем<пробел>следуэт<пробел> <пробел>гусев<пробел>и<пробел>маша<пробел>жили<пробел>в<пробел>одной<пробел>комнате<пробел>в<пробел>когда<пробел>то<пробел>роскошном<пробел>огромном<пробел>тепеь<пробел>у<пробел>заброшенном<пробел>доме<пробел> <пробел>в<пробел>гладких<пробел>стена<пробел>узкие<пробел>как<пробел>щели<пробел>глубокие<пробел>отверстия<пробел>рукону<пробел> <пробел>до<пробел>сумерек<пробел>переливался<пробел>гувор<пробел>шли<пробел>виразные<пробел>разговорыв<пробел>полпе<пробел>жидающей<пробел>необыкноенного<пробел>собыки<пробел> <пробел>небольшая<пробел>дверца<пробел>в<пробел>нём<пробел>была<пробел>приотворена<пробел>на<пробел>пороге<пробел>сиделс<пробел>на<пробел>корточках<пробел><пробел>рабочий<пробел>и<пробел>размешивал<пробел>ведёрке<пробел>киритичнокрасный<пробел>сурик<пробел> <пробел>неужеле<пробел>ты<пробел>невидел<пробел>лицо<пробел>того<пробел>мальчика<пробе

(227, 14.93095, 0.052637734)
<пробел>его<пробел>ярко<пробел>освещённая<пробел>внутренность<пробел>з<пробел>стёганой<пробел>ромбами<пробел>жёлтой<пробел>кожи<пробел>былавидна<пробел>сквозь<пробел>кругое<пробел>пверстие<пробел>люкоа<пробел> <пробел>аппарат<пробел>прорезивал<пробел>облака<пробел>над<пробел>тусклой<пробел>равниной<пробел>и<пробел>ревя<пробел>и<пробел>сотрясаясь<пробел>медлено<пробел>теперь<пробел>о<пробел>пускался<пробел>сагимся<пробел>успелтиль<пробел>кокрикнут<пробел>лосс<пробел>и<пробел>выключил<пробел>двигател<пробел> <пробел>он<пробел>открыл<пробел>двере<пробел>спальню<пробел>где<пробел>посе<пробел>смерти<пробел>кати<пробел>уон<пробел>никада<пробел>не<пробел>ночевал <пробел>что<пробел>заставляит<пробел>тебя<пробел>иди<пробел>насерть<пробел> <пробел>покамест<пробел>ему<пробел>всиэто<пробел>подавалосьи<пробел>разабрито<пробел>и<пробел>просды<пробел>холодно<пробел>он<пробел>заставил<пробел>слогу<пробел>или<пробел>полового<пробел> <пробел>над<пробел>головой<пробел>етела<п

(231, 14.752647, 0.060831767)
<пробел>ссамоваромизкрасный<пробел>меде<пробел>илицо<пробел>так<пробел>же<пробел>красны<пробел>как<пробел>самавар<пробел> <пробел>изпод<пробел>ног<пробел>выбегали<пробел>животны<пробел>похожие<пробел>на<пробел>каменных<пробел>ящириц<пробел>ярко<пробел>оранжевые<пробел>с<пробел>зубчатыми<пробел>хребтом<пробел> <пробел>движениим<пробел>бравей<пробел>тускуб<пробел>заставил<пробел>утихнуть<пробел>амфитеатр<пробел>историе<пробел>марса<пробел>оконщена<пробел> <пробел>несколико<пробел>раз<пробел>в<пробел>гуще<пробел>лапчатой<пробел>заросли<пробел>скользили<пробел>кидались<пробел>сторону<пробел>кикието<пробел>щетинистые<пробел>клубки<пробел> <пробел>в<пробел>деревюехть<пробел>ротес<пробел>натерью<пробел>пормерли<пробел>баратьяя<пробел>убты<пробел>земля<пробел>заброшена<пробел> <пробел>что<пробел>у<пробел>этой<пробел>барышни<пробел>было<пробел>в<пробел>голове<пробел>не<пробел>пойму<пробел>до<пробел>сих<пробел>пор<пробел> <пробел>отсюда<пробел>была<пробел>видна<проб

Если у Вас мощный компьютер, можно обучиться на полных данных

In [ ]:
num_epochs=1000
epoch_save_step=5
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint1")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X, y, 4):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,

                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)